In [2]:
!pip install scikit-learn xport numpy pandas

    100% |████████████████████████████████| 5.5MB 184kB/s ta 0:00:01
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/1f/c7/198496417c9c2f6226616cff7dedf2115a4f4d0276613bab842ec8ac1e23/numpy-1.16.4-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/db/83/7d4008ffc2988066ff37f6a0bb6d7b60822367dcb36ba5e39aa7801fda54/pandas-0.24.2-cp27-cp27mu-manylinux1_x86_64.whl
    100% |████████████████████████████████| 24.8MB 50kB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/3d/73/fe30c2daaaa0713420d0382b16fbb761409f532c56bdcc514bf7b6262bb6/pytz-2019.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a72

In [1]:
from os import listdir
from os.path import isfile, join
# import xport
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, fbeta_score

from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
data_path = "Data/"
data_files = [join(data_path, f) for f in listdir(data_path) if isfile(join(data_path, f))] #
data_files

['Data/DIQ_C.XPT',
 'Data/DIQ_I.XPT',
 'Data/DIQ_G.XPT',
 'Data/DIQ_B.XPT',
 'Data/DIQ_H.XPT',
 'Data/DIQ_F.XPT']

In [3]:
n_file = 1
data_path_latest = 'Data/DIQ_I.XPT' #choix du fichier de travail
df = pd.read_sas(data_path_latest)
print(data_files[-1], df.shape)

na = float(0)
df = df.fillna(value=na) #missing values

df = df.drop(['DIQ010', 'DID040', 'DIQ175E', 'DIQ172', 'DIQ175A', 'DIQ175B', 'DIQ175C', 'DIQ175D', 'DIQ175F', 'DIQ175G', 'DIQ175G', 'DIQ175H', 'DIQ175I', 'DIQ175J', 'DIQ175K', 'DIQ175L', 'DIQ175M', 'DIQ175N', 'DIQ175O', 'DIQ175P', 'DIQ175Q', 'DIQ175R', 'DIQ175S', 'DIQ175T', 'DIQ175U', 'DIQ175V', 'DIQ175W', 'DIQ175X', 'DID060', 'DIQ060U',
       'DIQ070', 'DIQ280', 'DIQ291', 'DID330', 'DIQ080'], axis=1)
#df.head()
df.columns

#clean des datas fini

Data/DIQ_F.XPT (9575, 54)


Index(['SEQN', 'DIQ160', 'DIQ170', 'DIQ180', 'DIQ050', 'DIQ230', 'DIQ240',
       'DID250', 'DID260', 'DIQ260U', 'DIQ275', 'DIQ300S', 'DIQ300D',
       'DID310S', 'DID310D', 'DID320', 'DID341', 'DID350', 'DIQ350U',
       'DIQ360'],
      dtype='object')

In [12]:
#définition des X et des Y

X = df[['DIQ170', 'DIQ180', 'DIQ050', 'DIQ230', 'DIQ240',
       'DID250', 'DID260', 'DIQ260U', 'DIQ275', 'DIQ300S', 'DIQ300D',
       'DID310S', 'DID310D', 'DID320', 'DID341', 'DID350', 'DIQ350U',
       'DIQ360']]
Y = np.array(df[["DIQ160"]] > 0)

print(X.shape)
print(Y.shape)

df['DIQ160'].value_counts()

(9575, 18)
(9575, 1)


2.0    5521
0.0    3530
1.0     513
9.0      11
Name: DIQ160, dtype: int64

In [13]:
#On a un pb : y'a bcp plus de y=0 (pas prediabtiques) que de y=1 (prediabetiques).
# On met en place des poids
zero=float(0)
one=float(1)

(Y == one).sum() / len(Y)
w0, w1 = (Y == zero).sum() / len(Y), (Y > zero).sum() / len(Y)

X, Y = np.array(X), np.array(Y)
df.head()

,SEQN,DIQ160,DIQ170,DIQ180,DIQ050,DIQ230,DIQ240,DID250,DID260,DIQ260U,DIQ275,DIQ300S,DIQ300D,DID310S,DID310D,DID320,DID341,DID350,DIQ350U,DIQ360
0,83732.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,1.0,1.0,1.0,134.0,72.0,6666.0,6666.0,9999.0,5.397605e-79,1.0,1.0,2.0
1,83733.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
2,83734.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,2.0,2.0,1.0,141.0,68.0,140.0,60.0,9999.0,2.000000e+00,2.0,2.0,2.0
3,83735.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
4,83736.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0


In [16]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, train_size=0.95, test_size=0.05)

print("Loading complete.")

# Define the models
model = RandomForestClassifier(
    n_estimators=50,
    random_state=0,
    class_weight={0: w1, 1: w0}
)

print("Model defined.")

model.fit(X_train, y_train) #Entrainer l'algo

print("Algorithm trained.")

preds = model.predict(X_valid) #Le faire tourner sur des données jamais vues

print("Model score (accuracy):")
print(model.score(X_valid, y_valid))
print("F-1 score: {}".format(fbeta_score(y_valid, preds, 1)))

Loading complete.
Model defined.
Algorithm trained.
Model score (accuracy):
0.9853862212943633
F-1 score: 0.9876977152899824


/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
